In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


### Создадим движок для postgresql
engine = create_engine('postgresql://hidden')

In [2]:
query = '''SELECT *,
                 MIN(date) OVER (PARTITION BY package_id) AS first_purchase_date,
                 MAX(rn_package_id) OVER (PARTITION BY package_id) AS count_pays
                 
          FROM level_2_analytical_tables.transactions
          WHERE fractional > 0 AND fractional < 1 
          '''

In [3]:
transaction = pd.read_sql_query(query, engine)

- группировка по дате первой покупки и сумма всех транзакций = фактическая выручка 
- группировка по дате первой покупки и сумма стоимости пакета с фильтром на rn_pack = 1 = потенциальная выручка

**Выручка по месячно**

----------

In [4]:
def transform_months_pay_rr_data(data):

    
    # Создаю признак когорты меясяца
    data['first_purchase_month'] = data['first_purchase_date'].dt.to_period('M').dt.to_timestamp()

    
    
    # Считаю фактическую выручку из суммы транзакций 
    fact_money = data.groupby(['first_purchase_month', 'first_sale'])\
                     .agg({'amount':'sum'})\
                     .sort_values('first_purchase_month', ascending = False)\
                     .reset_index()


    # Считаю потенциальную выручку из общей стоимости пакета
    potencial_money = data[data['rn_package_id'] == 1].groupby(['first_purchase_month', 'first_sale'])\
                                                      .agg({'amount_full': ['sum', 'count']})\
                                                      .sort_values('first_purchase_month', ascending=False)\
                                                      .reset_index()

    
    # Переименование столбцов по кол-ву оплаты исходя из кол-ва транзакции по пакетам рр
    potencial_money.columns = ['first_purchase_month', 'first_sale', 'sum_amount_full', 'count_amount_full']



    four_pays = data[data['count_pays'] == 4].groupby(['first_purchase_month', 'first_sale'])\
                                             .agg({'package_id': 'nunique'})\
                                             .sort_values('first_purchase_month', ascending=False)\
                                             .reset_index()\
                                             .rename(columns = {'package_id':'4_pays'})

    three_pays = data[data['count_pays'] == 3].groupby(['first_purchase_month', 'first_sale'])\
                                              .agg({'package_id': 'nunique'})\
                                              .sort_values('first_purchase_month', ascending=False)\
                                              .reset_index()\
                                              .rename(columns = {'package_id':'three_pays'})


    two_pays = data[data['count_pays'] == 2].groupby(['first_purchase_month', 'first_sale'])\
                                            .agg({'package_id': 'nunique'})\
                                            .sort_values('first_purchase_month', ascending=False)\
                                            .reset_index()\
                                            .rename(columns = {'package_id':'two_pays'})

    one_pays = data[data['count_pays'] == 1].groupby(['first_purchase_month', 'first_sale'])\
                                                          .agg({'package_id': 'nunique'})\
                                                          .sort_values('first_purchase_month', ascending=False)\
                                                          .reset_index()\
                                                          .rename(columns = {'package_id':'one_pays'})



    # Обединение дф'ов и расчет процентов 
    merged_data = fact_money.merge(potencial_money, on = ['first_purchase_month', 'first_sale'])
    merged_data['percent_fact_of_potencial'] = round(merged_data['amount'] / merged_data['sum_amount_full']*100,2)

    merged_data = merged_data.merge(four_pays, on = ['first_purchase_month', 'first_sale'], how = 'left')
    merged_data['percent_end_buy_of_all'] = round(merged_data['4_pays'] / merged_data['count_amount_full']*100,2)

    merged_data = merged_data.merge(three_pays, on = ['first_purchase_month', 'first_sale'], how = 'left')
    merged_data['percent_three_buy_of_all'] = round(merged_data['three_pays'] / merged_data['count_amount_full']*100,2)

    merged_data = merged_data.merge(two_pays, on = ['first_purchase_month', 'first_sale'], how = 'left')
    merged_data['percent_two_buy_of_all'] = round(merged_data['two_pays'] / merged_data['count_amount_full']*100,2)

    merged_data = merged_data.merge(one_pays, on = ['first_purchase_month', 'first_sale'], how = 'left')
    merged_data['percent_one_buy_of_all'] = round(merged_data['one_pays'] / merged_data['count_amount_full']*100,2)
    
    
    
    # Функция для преобразования типа в числовое значение
    def type_sale_to_words(type_value):
        return 'первичные' if type_value == 1 else 'вторичные'
    
    

    # Создание новой колонки 'тип_числовой' на основе 'тип'
    merged_data['first_sale_full'] = merged_data['first_sale'].apply(type_sale_to_words)
    
    
    
    # Считаю сколько осталось выплатить по рассрочке 
    merged_data['осталось_выплатить'] = np.where(
    np.isnan(merged_data['sum_amount_full']),
    merged_data['sum_amount_full'],
    merged_data['sum_amount_full'] - merged_data['amount'])
    
    
    
    # Считаю сколько осталось невыплаченных пакетов по рассрочке
    merged_data['осталось_выплатить_количество_продаж'] = np.where(
    np.isnan(merged_data['4_pays']),
    merged_data['count_amount_full'],
    merged_data['count_amount_full'] - merged_data['4_pays'])

    merged_data = merged_data[[  'first_purchase_month',
                                 'first_sale_full',
                                 'sum_amount_full',
                                 'amount',
                                 'percent_fact_of_potencial',
                                 'count_amount_full',
                                 '4_pays',
                                 'percent_end_buy_of_all',
                                 'three_pays',
                                 'percent_three_buy_of_all',
                                 'two_pays',
                                 'percent_two_buy_of_all',
                                 'one_pays',
                                 'percent_one_buy_of_all',
                                 'осталось_выплатить_количество_продаж',
                                 'осталось_выплатить'
                                 ]]
    
    merged_data = merged_data.rename(columns={'first_purchase_month':'когорта_месяца',
                                             'first_sale_full' :'признак_продажи',
                                             'sum_amount_full' :'полная_цена_пакетов',
                                             'amount' :'сумма_траназкции',
                                             'percent_fact_of_potencial':'процент_фактической_от_потенциальной',
                                             'count_amount_full' :'количество_пакетов_общее',
                                             '4_pays' :'4_транзакции_по_рр',
                                             'percent_end_buy_of_all' :'процент_пакетов_с_4_транзак_от_всех_пакетов',
                                             'three_pays' :'3_транзакции_по_рр',
                                             'percent_three_buy_of_all' :'процент_пакетов_с_3_транзак_от_всех_пакетов',
                                             'two_pays' :'2_транзакции_по_рр',
                                             'percent_two_buy_of_all' :'процент_пакетов_с_2_транзак_от_всех_пакетов',
                                             'one_pays' :'1_транзакции_по_рр',
                                             'percent_one_buy_of_all' :'процент_пакетов_с_1_транзак_от_всех_пакетов',
                                             })
    
    merged_data = merged_data[merged_data['когорта_месяца'] >= '2023-05']
    
    return merged_data



In [5]:
transaction_month = transform_months_pay_rr_data(transaction)
transaction_month

,когорта_месяца,признак_продажи,полная_цена_пакетов,сумма_траназкции,процент_фактической_от_потенциальной,количество_пакетов_общее,4_транзакции_по_рр,процент_пакетов_с_4_транзак_от_всех_пакетов,3_транзакции_по_рр,процент_пакетов_с_3_транзак_от_всех_пакетов,2_транзакции_по_рр,процент_пакетов_с_2_транзак_от_всех_пакетов,1_транзакции_по_рр,процент_пакетов_с_1_транзак_от_всех_пакетов,осталось_выплатить_количество_продаж,осталось_выплатить
0,2024-02-01,первичные,396270.0,123052.0,31.05,14,NaN,NaN,NaN,NaN,3.0,21.43,11.0,78.57,14.0,273218.0
1,2024-02-01,вторичные,52480.0,18741.0,35.71,2,NaN,NaN,NaN,NaN,1.0,50.00,1.0,50.00,2.0,33739.0
2,2024-01-01,первичные,485690.0,312291.0,64.30,13,2.0,15.38,5.0,38.46,4.0,30.77,2.0,15.38,11.0,173399.0
3,2024-01-01,вторичные,104350.0,39576.0,37.93,5,NaN,NaN,1.0,20.00,1.0,20.00,3.0,60.00,5.0,64774.0
4,2023-12-01,первичные,1641820.0,1351731.0,82.33,54,34.0,62.96,5.0,9.26,7.0,12.96,10.0,18.52,20.0,290089.0
5,2023-12-01,вторичные,137860.0,55451.0,40.22,5,1.0,20.00,NaN,NaN,1.0,20.00,3.0,60.00,4.0,82409.0
6,2023-11-01,первичные,6267370.0,4821482.0,76.93,196,116.0,59.18,15.0,7.65,19.0,9.69,46.0,23.47,80.0,1445888.0
7,2023-11-01,вторичные,536610.0,417429.0,77.79,16,11.0,68.75,2.0,12.50,NaN,NaN,3.0,18.75,5.0,119181.0
8,2023-10-01,первичные,5082280.0,4083664.0,80.35,138,95.0,68.84,7.0,5.07,10.0,7.25,26.0,18.84,43.0,998616.0
9,2023-10-01,вторичные,320730.0,221506.0,69.06,9,4.0,44.44,1.0,11.11,2.0,22.22,2.0,22.22,5.0,99224.0


In [30]:
transaction_month.to_sql(
    name='months_rr_analyze',
    con=engine,
    if_exists='replace',
    index=False,
    schema='level_2_analytical_tables',
    chunksize=10000
)

16

-----------